Bộ dữ liệu có các thuộc tính liên quan đến hoạt động của máy móc và thông số kỹ thuật trên dây chuyền sản xuất. Một số thuộc tính có thể bao gồm:


*   Date: Ngày ghi nhận dữ liệu
*   Các thông số kỹ thuật khác: Có thể là các thông số như áp suất, nhiệt độ, rung động, tốc độ, v.v.
*   Downtime_No_Machine_Failure: Cột mục tiêu đại diện cho trạng thái máy ngừng hoạt động nhưng không có lỗi máy.

Bài toán này liên quan đến việc quản lý và phân tích thông tin ngừng hoạt động của máy móc để dự đoán trạng thái không có lỗi dựa trên các thông số khác.

 Mô tả bài toán:

  Bài toán code này nhằm mục tiêu xây dựng một mô hình học máy để phân loại trạng thái Downtime_No_Machine_Failure. Các bước thực hiện gồm:

* Đọc và xử lý dữ liệu, xử lý các giá trị bị thiếu.
* Trích xuất các đặc trưng từ dữ liệu, bao gồm tách cột ngày tháng và tạo các cột Day, Month, và Year.
* Huấn luyện mô hình phân loại Naive Bayes (GaussianNB).
* Đánh giá hiệu suất của mô hình qua các thước đo như độ chính xác, báo cáo phân loại, và ma trận nhầm lẫn.



In [ ]:
pip install pandas scikit-learn

Cài đặt các thư viện pandas và scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Import các thư viện cần thiết cho việc xử lý dữ liệu

In [ ]:
data = pd.read_csv('/content/MachineDowntime.csv')

In [ ]:
print(data.dtypes)

Hydraulic_Pressure(bar)            float64
Coolant_Pressure(bar)              float64
Air_System_Pressure(bar)           float64
Coolant_Temperature                float64
Hydraulic_Oil_Temperature(?C)      float64
Spindle_Bearing_Temperature(?C)    float64
Spindle_Vibration(?m)              float64
Tool_Vibration(?m)                 float64
Spindle_Speed(RPM)                 float64
Voltage(volts)                     float64
Torque(Nm)                         float64
Cutting(kN)                        float64
Day                                  int32
Month                                int32
Year                                 int32
Machine_ID_Makino-L2-Unit1-2015       bool
Machine_ID_Makino-L3-Unit1-2015       bool
Assembly_Line_No_Shopfloor-L2         bool
Assembly_Line_No_Shopfloor-L3         bool
Downtime_No_Machine_Failure           bool
dtype: object


In [ ]:
for column in data.columns:
    if data[column].isnull().sum() > 0:
        data[column].fillna(data[column].mean(), inplace=True)

Duyệt qua các cột trong data để tìm cột nào có giá trị bị thiếu và điền các giá trị bị thiếu bằng giá trị trung bình của cột đó.

In [ ]:
target_column = 'Downtime_No_Machine_Failure'

In [ ]:
if target_column in data.columns:
    X = data.drop(target_column, axis=1)
    y = data[target_column]
else:
    print(f"Cột '{target_column}' không tồn tại trong DataFrame.")


Xác định cột mục tiêu là Downtime_No_Machine_Failure. Nếu cột này tồn tại, tách nó ra khỏi X (dữ liệu huấn luyện) và y (nhãn mục tiêu).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 80/20, đảm bảo tính ngẫu nhiên bằng random_state=42.

In [ ]:
if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'], format='%d-%m-%Y')  # Chuyển đổi thành kiểu datetime
    data['Day'] = data['Date'].dt.day
    data['Month'] = data['Date'].dt.month
    data['Year'] = data['Date'].dt.year
    data.drop('Date', axis=1, inplace=True)

Nếu cột Date có trong dữ liệu, chuyển đổi cột này thành kiểu datetime, sau đó trích xuất Day, Month, và Year, rồi loại bỏ cột Date ban đầu.

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

Tạo mô hình Naive Bayes (GaussianNB) và huấn luyện nó với tập dữ liệu huấn luyện.

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print("Đánh giá mô hình:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Đánh giá mô hình:
Accuracy: 0.84
Classification Report:
               precision    recall  f1-score   support

       False       0.82      0.84      0.83       231
        True       0.86      0.84      0.85       269

    accuracy                           0.84       500
   macro avg       0.84      0.84      0.84       500
weighted avg       0.84      0.84      0.84       500

Confusion Matrix:
 [[195  36]
 [ 44 225]]


Dự đoán nhãn cho tập kiểm tra và in ra các đánh giá về mô hình bao gồm độ chính xác, báo cáo phân loại, và ma trận nhầm lẫn để hiểu rõ hơn về hiệu suất của mô hình.